In [ ]:
!pip install openpyxl
!pip install xlsxwriter

In [1]:
import pandas as pd
import openpyxl
import xlsxwriter
from datetime import date



In [41]:
input_df = pd.read_excel('../sample_files/Input_SP_Auto.xlsx',index_col=False)
input_df

,SKU,CODE,Market,PPC Type,Targeting type,Match type,BRAND,Date,PIC,STT,Budget,Targeting,Bid,Bid strategy,Placement,Percentage,Portfolio
0,set 9 - 001,CC-0031-0002,US,SP,KW,EXACT,Chicnchill,20221212,HUY,1,10,"abc123,cde456",0.5,Fixed Bid,placementTop,0.1,4567
1,set 9 - 001,CC-0031-0002,US,SP,ASIN,EXACT,Chicnchill,20221212,HUY,2,10,"abc123,cde456","0.5,0.7",Fixed Bid,placementTop,0.1,4567


<h2>SP Setup</h2>

In [33]:
# Create Setup DataFrame

def create_single_row_SP(columns:list,cam_id):
    single_row_dictionary = {}
    for item in columns:
        if item == 'Product':
            single_row_dictionary[item] = 'Sponsored Products'
        elif item == 'Operation':
            single_row_dictionary[item] = 'Create'
        elif item == 'Campaign Id':
            single_row_dictionary[item] = cam_id
        elif item == 'State':
            single_row_dictionary[item] = 'enabled'
        else:
            single_row_dictionary[item] = None
    return single_row_dictionary

def createDataFrameSP_asin(cam_id,asin_list:list,budget,productName,bid_list:list,billing_strategy,placement,percentage):
    columns=['Product','Entity','Operation','Campaign Id','Ad Group Id','Portfolio Id',
                           'Ad Id','Keyword Id','Product Targeting Id','Campaign Name','Ad Group Name',
                           'Start Date','End Date','Targeting Type','State','Daily Budget','sku','asin',
                           'Ad Group Default Bid','Bid','Keyword Text','Match Type','Bidding Strategy',
                           'Placement','Percentage','Product Targeting Expression']
    df = pd.DataFrame(columns=columns)

# 5 dong co dinh
    for i in range (0,5):
        df = df.append(create_single_row_SP(columns=columns,cam_id=cam_id),ignore_index=True)
        
    df.iloc[0]['Entity'] = 'Campaign'
    df.iloc[0]['Campagin Name'] = cam_id
    df.iloc[0]['Targeting Type'] = 'Manual'
    df.iloc[0]['Start Date'] = date.today().strftime("%Y%m%d")
    df.iloc[0]['Daily Budget'] = budget
    df.iloc[0]['Bidding Strategy'] = billing_strategy

    df.iloc[1]['Entity'] = 'Bidding Adjustment'
    df.iloc[1]['Placement'] = placement
    df.iloc[1]['Percentage'] = percentage

    df.iloc[2]['Entity'] = 'Bidding Adjustment'
    df.iloc[2]['Placement'] = placement
    df.iloc[2]['Percentage'] = percentage

    df.iloc[3]['Entity'] = 'Ad group'
    df.iloc[3]['Ad Group Id'] = cam_id
    df.iloc[3]['Ad Group Name'] = cam_id
    # df.iloc[3]['Ad Group Default Bid'] = bid
    
    df.iloc[4]['Entity'] = 'Product ad'
    df.iloc[4]['Ad Group Id'] = cam_id
    df.iloc[4]['sku'] = productName


# Optional Asin - so dong Asin
    for i in range(len(asin_list)):
        asin_row = create_single_row_SP(columns=columns,cam_id=cam_id)
        asin_row['Entity'] = 'Product targeting'
        asin_row['Ad Group Id'] = cam_id
        if len(bid_list)>1:
            if i < len(bid_list):
                asin_row['Bid'] = bid_list[i]
            else:
                asin_row['Bid'] = bid_list[-1]
        else:
            asin_row['Bid'] = bid_list[0]
        asin_row['Product Targeting Expression'] = '''asin="''' +str(asin_list[i])+ '''"'''
        df = df.append(asin_row,ignore_index=True)
# Optional Asin - so dong Asin  
    # for item in asin_list:
    #     asin_row = create_single_row(columns=columns,cam_id=cam_id)
    #     asin_row['Entity'] = 'Product targeting'
    #     asin_row['Ad Group Id'] = cam_id
        
    #     asin_row['Product Targeting Expression'] = '''asin="''' +str(item)+ '''"'''
    #     df = df.append(asin_row,ignore_index=True)
    
    return df

In [34]:
# Create DataFrame - Case Keyword
def createDataFrameSP_keyword(cam_id,keyword_list:list,budget,productName,bid,billing_strategy,portfolio_id,match_type):
    columns=['Product','Entity','Operation','Campaign Id','Ad Group Id','Portfolio Id',
                           'Ad Id','Keyword Id','Product Targeting Id','Campaign Name','Ad Group Name',
                           'Start Date','End Date','Targeting Type','State','Daily Budget','sku','asin',
                           'Ad Group Default Bid','Bid','Keyword Text','Match Type','Bidding Strategy',
                           'Placement','Percentage','Product Targeting Expression']
    df = pd.DataFrame(columns=columns)

# 5 dong co dinh
    for i in range (0,5):
        df = df.append(create_single_row_SP(columns=columns,cam_id=cam_id),ignore_index=True)
        
    df.iloc[0]['Entity'] = 'Campaign'
    df.iloc[0]['Campagin Name'] = cam_id
    df.iloc[0]['Targeting Type'] = 'Manual'
    df.iloc[0]['Start Date'] = date.today().strftime("%Y%m%d")
    df.iloc[0]['Daily Budget'] = budget
    df.iloc[0]['Bidding Strategy'] = billing_strategy
    df.iloc[0]['Portfolio Id'] = portfolio_id

    df.iloc[1]['Entity'] = 'Bidding Adjustment'
    df.iloc[1]['Placement'] = 'placementTop'
    df.iloc[1]['Percentage'] = '0%'

    df.iloc[2]['Entity'] = 'Bidding Adjustment'
    df.iloc[2]['Placement'] = 'placementProductPage'
    df.iloc[2]['Percentage'] = '0%'

    df.iloc[3]['Entity'] = 'Ad group'
    df.iloc[3]['Ad Group Id'] = cam_id
    df.iloc[3]['Ad Group Name'] = cam_id
    df.iloc[3]['Ad Group Default Bid'] = bid
    
    df.iloc[4]['Entity'] = 'Product ad'
    df.iloc[4]['Ad Group Id'] = cam_id
    df.iloc[4]['sku'] = productName
    df.iloc[4]['Bid'] = bid

# Optional Product Targetting- so dong Keyword
    for item in keyword_list:
        keyword_row = create_single_row_SP(columns=columns,cam_id=cam_id)
        keyword_row['Entity'] = 'Product targeting'
        keyword_row['Ad Group Id'] = cam_id
        keyword_row['Keyword Text'] = str(item)
        keyword_row['Match Type'] = match_type
        df = df.append(keyword_row,ignore_index=True)
    
    return df

In [35]:
# output_df = createDataFrame_asin(x1,x2,x3,x4,x5)
# output_df"
output_keyword_df = createDataFrameSP_keyword(cam_id='ASD123456',keyword_list=['Khai1','Khai2'],budget=2.9,productName='Newspaper',bid=0.6,billing_strategy='Fixed_bid',portfolio_id='5555555555',match_type='Exact')
output_keyword_df

,Product,Entity,Operation,Campaign Id,Ad Group Id,Portfolio Id,Ad Id,Keyword Id,Product Targeting Id,Campaign Name,...,sku,asin,Ad Group Default Bid,Bid,Keyword Text,Match Type,Bidding Strategy,Placement,Percentage,Product Targeting Expression
0,Sponsored Products,Campaign,Create,ASD123456,None,5555555555,None,None,None,None,...,None,None,None,None,None,None,Fixed_bid,None,None,None
1,Sponsored Products,Bidding Adjustment,Create,ASD123456,None,None,None,None,None,None,...,None,None,None,None,None,None,None,placementTop,0%,None
2,Sponsored Products,Bidding Adjustment,Create,ASD123456,None,None,None,None,None,None,...,None,None,None,None,None,None,None,placementProductPage,0%,None
3,Sponsored Products,Ad group,Create,ASD123456,ASD123456,None,None,None,None,None,...,None,None,0.6,None,None,None,None,None,None,None
4,Sponsored Products,Product ad,Create,ASD123456,ASD123456,None,None,None,None,None,...,Newspaper,None,None,0.6,None,None,None,None,None,None
5,Sponsored Products,Product targeting,Create,ASD123456,ASD123456,None,None,None,None,None,...,None,None,None,None,Khai1,Exact,None,None,None,None
6,Sponsored Products,Product targeting,Create,ASD123456,ASD123456,None,None,None,None,None,...,None,None,None,None,Khai2,Exact,None,None,None,None


In [31]:
output_asin_df = createDataFrameSP_asin(cam_id='ASD123456',asin_list=['Khai1','Khai2','Khai3'],budget=2.9,productName='Newspaper',bid_list=[0.6,0.7],billing_strategy='Fixed_bid')
output_asin_df

,Product,Entity,Operation,Campaign Id,Ad Group Id,Portfolio Id,Ad Id,Keyword Id,Product Targeting Id,Campaign Name,...,sku,asin,Ad Group Default Bid,Bid,Keyword Text,Match Type,Bidding Strategy,Placement,Percentage,Product Targeting Expression
0,Sponsored Products,Campaign,Create,ASD123456,None,None,None,None,None,None,...,None,None,None,NaN,None,None,Fixed_bid,None,None,None
1,Sponsored Products,Bidding Adjustment,Create,ASD123456,None,None,None,None,None,None,...,None,None,None,NaN,None,None,None,placementTop,0%,None
2,Sponsored Products,Bidding Adjustment,Create,ASD123456,None,None,None,None,None,None,...,None,None,None,NaN,None,None,None,placementProductPage,0%,None
3,Sponsored Products,Ad group,Create,ASD123456,ASD123456,None,None,None,None,None,...,None,None,None,NaN,None,None,None,None,None,None
4,Sponsored Products,Product ad,Create,ASD123456,ASD123456,None,None,None,None,None,...,Newspaper,None,None,NaN,None,None,None,None,None,None
5,Sponsored Products,Product targeting,Create,ASD123456,ASD123456,None,None,None,None,None,...,None,None,None,0.6,None,None,None,None,None,"asin=""Khai1"""
6,Sponsored Products,Product targeting,Create,ASD123456,ASD123456,None,None,None,None,None,...,None,None,None,0.7,None,None,None,None,None,"asin=""Khai2"""
7,Sponsored Products,Product targeting,Create,ASD123456,ASD123456,None,None,None,None,None,...,None,None,None,0.7,None,None,None,None,None,"asin=""Khai3"""


In [32]:
# df.to_excel('myexcel.xlsx',sheet_name='Sheet1',engine='xlsxwriter')
writer = pd.ExcelWriter("dataframe2.xlsx", engine='xlsxwriter')
output_asin_df.to_excel(writer,sheet_name = 'sheet_asin', index=False)
# output_keyword_df.to_excel(writer,sheet_name = 'sheet_keyword', index=False)
writer.save() 


In [ ]:
workbook = xlsxwriter.Workbook('write_list.xlsx')
worksheet = workbook.sheet_name('sheet_keyword')


In [42]:
input_df

,SKU,CODE,Market,PPC Type,Targeting type,Match type,BRAND,Date,PIC,STT,Budget,Targeting,Bid,Bid strategy,Placement,Percentage,Portfolio
0,set 9 - 001,CC-0031-0002,US,SP,KW,EXACT,Chicnchill,20221212,HUY,1,10,"abc123,cde456",0.5,Fixed Bid,placementTop,0.1,4567
1,set 9 - 001,CC-0031-0002,US,SP,ASIN,EXACT,Chicnchill,20221212,HUY,2,10,"abc123,cde456","0.5,0.7",Fixed Bid,placementTop,0.1,4567


In [43]:
dataframes_list =[] 
for i in range(len(input_df)):
    cam_id = str(input_df.iloc[i]['CODE']) +  str(input_df.iloc[i]['Market']) + str(input_df.iloc[i]['PPC Type']) + str(input_df.iloc[i]['Match type'])+ str(input_df.iloc[i]['BRAND']) + str(input_df.iloc[i]['Date']) +str(input_df.iloc[i]['PIC'] + str(input_df.iloc[i]['STT']))
    kw_as_list = input_df.iloc[i]['Targeting'].split(',')
    if input_df.iloc[i]['Targeting type'] == 'KW':
        kw_df = createDataFrameSP_keyword(cam_id=cam_id,keyword_list=kw_as_list,budget=input_df.iloc[i]['Budget'],productName=input_df.iloc[i]['SKU'],bid=str(input_df.iloc[i]['Bid']),billing_strategy=input_df.iloc[i]['Bid strategy'],portfolio_id=input_df.iloc[i]['Portfolio'],match_type=input_df.iloc[i]['Match type'])
        dataframes_list.append(kw_df)
    elif input_df.iloc[i]['Targeting type'] == 'ASIN':
        bid_list = input_df.iloc[i]['Bid']
        asin_df = createDataFrameSP_asin(cam_id=cam_id,asin_list=kw_as_list,budget=input_df.iloc[i]['Budget'],productName=input_df.iloc[i]['SKU'],bid_list=bid_list,billing_strategy=input_df.iloc[i]['Bid strategy'])
        dataframes_list.append(asin_df)
    
    result_df = pd.concat(dataframes_list)

    


In [44]:
result_df

,Product,Entity,Operation,Campaign Id,Ad Group Id,Portfolio Id,Ad Id,Keyword Id,Product Targeting Id,Campaign Name,...,sku,asin,Ad Group Default Bid,Bid,Keyword Text,Match Type,Bidding Strategy,Placement,Percentage,Product Targeting Expression
0,Sponsored Products,Campaign,Create,CC-0031-0002USSPEXACTChicnchill20221212HUY 1,None,4567,None,None,None,None,...,None,None,None,None,None,None,Fixed Bid,None,None,None
1,Sponsored Products,Bidding Adjustment,Create,CC-0031-0002USSPEXACTChicnchill20221212HUY 1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,placementTop,0%,None
2,Sponsored Products,Bidding Adjustment,Create,CC-0031-0002USSPEXACTChicnchill20221212HUY 1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,placementProductPage,0%,None
3,Sponsored Products,Ad group,Create,CC-0031-0002USSPEXACTChicnchill20221212HUY 1,CC-0031-0002USSPEXACTChicnchill20221212HUY 1,None,None,None,None,None,...,None,None,0.5,None,None,None,None,None,None,None
4,Sponsored Products,Product ad,Create,CC-0031-0002USSPEXACTChicnchill20221212HUY 1,CC-0031-0002USSPEXACTChicnchill20221212HUY 1,None,None,None,None,None,...,set 9 - 001,None,None,0.5,None,None,None,None,None,None
5,Sponsored Products,Product targeting,Create,CC-0031-0002USSPEXACTChicnchill20221212HUY 1,CC-0031-0002USSPEXACTChicnchill20221212HUY 1,None,None,None,None,None,...,None,None,None,None,abc123,EXACT,None,None,None,None
6,Sponsored Products,Product targeting,Create,CC-0031-0002USSPEXACTChicnchill20221212HUY 1,CC-0031-0002USSPEXACTChicnchill20221212HUY 1,None,None,None,None,None,...,None,None,None,None,cde456,EXACT,None,None,None,None
0,Sponsored Products,Campaign,Create,CC-0031-0002USSPEXACTChicnchill20221212HUY 2,None,None,None,None,None,None,...,None,None,None,None,None,None,Fixed Bid,None,None,None
1,Sponsored Products,Bidding Adjustment,Create,CC-0031-0002USSPEXACTChicnchill20221212HUY 2,None,None,None,None,None,None,...,None,None,None,None,None,None,None,placementTop,0%,None
2,Sponsored Products,Bidding Adjustment,Create,CC-0031-0002USSPEXACTChicnchill20221212HUY 2,None,None,None,None,None,None,...,None,None,None,None,None,None,None,placementProductPage,0%,None
